In [1]:
import os
import io
import random
import numpy as np
import pandas as pd
from PIL import Image
import autokeras as ak
import tensorflow as tf
from numpy import asarray
from tensorflow import keras
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from tensorflow.keras.callbacks import CSVLogger
from sklearn.datasets import load_files
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold

In [2]:
#create images array by selecting 5000 images from each domain and create csv with instances labels, label per domain
directory = ["knapsackImages","imagesselectedset","gcImages"]
images = []
instancesTotal = 0
total = 0
Label = 0
df = pd.DataFrame(columns = ['instance', 'Label'])
for i in directory:
    for filename in os.listdir(i):
        if instancesTotal < 5000:
            instance = os.path.join(i, filename)
            instancesTotal+=1
            total+=1
            df.loc[total] = [filename,Label]
            if instance.endswith('.jpg'):
                im = Image.open(instance).convert('L')
                pix = im.load()
                width, height = im.size
                pixel_values = list(im.getdata())
                data = asarray(im)
                images.append(data)
            
            
    instancesTotal = 0
    Label+=1
    
images2 = np.asarray(images)
images2 = images2.reshape((images2.shape[0], 32, 32))
df.to_csv('alldomains.csv', encoding='utf-8') 

In [ ]:
#read the labels
npdf = pd.read_csv('alldomains.csv')
print(npdf['Label'].value_counts())
source = npdf.Label

In [ ]:
X = images2.astype('float32')
X /= 255
y = source

In [5]:
tf_callbacks = [
    tf.keras.callbacks.CSVLogger("TSPimagesTraining.csv", separator=",", append=False),
    tf.keras.callbacks.TensorBoard(log_dir='./logs',profile_batch = 100000000),
]

# Autkeras Model

In [ ]:
from sklearn import metrics
from sklearn.metrics import accuracy_score,recall_score,precision_score
cv = KFold(n_splits=10, random_state=1, shuffle=True)
all_tests = []
all_predictions = []
for train_index, test_index in cv.split(X, y):
  #print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    clf = ak.ImageClassifier(overwrite= False, max_trials=1)
  # Supervised training of the model
    print("Start training")
    clf.fit(X_train, y_train, epochs=100, callbacks=tf_callbacks)

    print("Predictions on unseen data")
    predicted_y = clf.predict(X_test)
    all_tests.append(y_test)
    all_predictions.append(predicted_y)
    report = classification_report(y_test.astype(str), predicted_y.astype(str))
    print(report)

In [ ]:
predictions = []
for sublist in all_predictions:
    for item in sublist:
        predictions.append(int(item))

tests = []
for sublist in all_tests:
    for item in sublist:
        tests.append(item)

In [10]:
report = classification_report(tests,predictions)
print(report)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5000
           1       1.00      0.91      0.95      5000
           2       0.91      1.00      0.95      5000

    accuracy                           0.97     15000
   macro avg       0.97      0.97      0.97     15000
weighted avg       0.97      0.97      0.97     15000



In [11]:
from sklearn import metrics
from sklearn.metrics import accuracy_score,recall_score,precision_score
from sklearn.metrics import f1_score

print("Accuracy:",metrics.accuracy_score(predictions,tests))
precision = precision_score(predictions,tests,average="macro")
recall = recall_score(predictions,tests,average="macro")
f1_score = f1_score(predictions,tests,average="macro")
print('Precision: %.3f' % precision)
print('Recall: %.3f' % recall)
print('F1-score: %.3f' % f1_score)

Accuracy: 0.9681333333333333
Precision: 0.968
Recall: 0.971
F1-score: 0.968
